# Activity 4: Azure CLI - Azure SQL Database

#### <i>The Azure SQL Workshop - Module 2</i>

<p style="border-bottom: 1px solid lightgrey;"></p>

In this activity, you'll walk through getting set up with the Azure CLI, using a PowerShell notebook in ADS.  

> Note: in PowerShell notebooks, you can use the PowerShell Az module or the Azure CLI. In this activity, you'll explore the Azure CLI, but you should know there are similar commands available for the [PowerShell Az module](https://docs.microsoft.com/en-us/powershell/azure/new-azureps-module-az?view=azps-3.5.0).  


**Set up**   

0. You should have opened this file using Azure Data Studio. If you didn't, please refer to Module 2 Activity 3 in the main Module file to get set up.  
1. In the bar at the top of this screen, confirm or change the "Kernel" to **PowerShell**. This determines what language the code blocks in the file are. In this case, that language is PowerShell.  
2. Confirming the Kernel is **PowerShell**, for "Attach to", confirm it reads **localhost**.  

Now that you're set up, you should read the text cells and "Run" the code cells by selecting the play button that appears in the left of a code cell when you hover over it.  
> Some of the cells have been run before, this is just to show you the expected result from the testing of the labs. If you choose not to complete the labs/prerequisites, do not run any cells, just review the results.      


In [ ]:
# Log in to the Azure portal with the Azure account (email address) and password you're using for the workshop
# You may get an error initially, run again, and you should get a pop-up that directs you through authenticating
az login

In [2]:
# Specify your subscription for the workshop
# Should appear in the list of subscriptions output in previous cell under "id" 
az account set --subscription <subscription-ID>

In [4]:
# Specify your default subscription, resource group, and Azure SQL Database logical server
# Replace ID with your ID for the workshop
az configure --defaults group=azuresqlworkshopID sql-server=aw-serverID

Now that you've got your details plugged in, let's see what you can do with the Azure CLI.

In [ ]:
# Show all databases in the Azure SQL Database logical server
az sql db list

There's a lot of information above. What if we just want to see specifics for the AdventureWorksID database?
>Note: Replace ID in the command with your unique ID you've been using for the workshop.

In [ ]:
az sql db show --name AdventureWorksID

You can also determine the database size and usage.

In [7]:
az sql db list-usages --name AdventureWorksID

[
  {
    "currentValue": 12582912.0,
    "displayName": "Database Size",
    "limit": 34359738368.0,
    "name": "database_size",
    "nextResetTime": null,
    "resourceName": "AdventureWorks168563",
    "unit": "Bytes"
  },
  {
    "currentValue": 16777216.0,
    "displayName": "Database Allocated Size",
    "limit": 34359738368.0,
    "name": "database_allocated_size",
    "nextResetTime": null,
    "resourceName": "AdventureWorks168563",
    "unit": "Bytes"
  }
]


You've now seen a few examples for how the `az sql db` commands work. There are also commands related to the Azure SQL Database logical server, and these fall under `az sql server`.  

>There are also similar commands for `az sql mi` and `az sql midb` (commands for databases within a Managed Instance, sometimes referred to as "Managed Databases").  

For detailed explanations of all the commands available, refer to the [Azure CLI documentation](https://docs.microsoft.com/en-us/cli/azure/?view=azure-cli-latest).

## Example: Managing connection policies

One thing thing that you might use the Azure CLI or Azure PowerShell commands for is updating the connection policy. We'll use this as an example for how you can manage Azure SQL using a tool like the Azure CLI. In this example, you'll look at Azure SQL Database and its commands, but the implementation is very similar in Azure SQL Managed Instance.  



### Overview: Connection policies

During deployment, in Azure SQL MI you're able to choose the connection type or policy. In Azure SQL DB, you can choose the connection type, but only via PowerShell or Azure CLI after deployment. You can keep the default (Proxy for connections from outside and Redirect for connections within Azure) or something else. At the highest level, in Proxy mode, all connections are proxied through the Azure SQL Database gateways, but in Redirect mode, after the connection is established leveraging the gateway, the connection is directly to the database or managed instance. The direct connection (redirect) allows for reduced latency and improved throughput, but also requires opening up additional ports (allowing inbound and outbound communication in the range of 11000 - 11999).    

> More information on connectivity for Azure SQL DB can be found [here](https://docs.microsoft.com/en-us/azure/sql-database/sql-database-connectivity-architecture) and for Azure SQL MI [here](https://docs.microsoft.com/en-us/azure/sql-database/sql-database-managed-instance-connectivity-architecture).  

Let's see what the current policy is using the Azure CLI.

In [ ]:
az sql server conn-policy show

So the results above tell us the connection type is "Default". Let's set it to "Proxy" and determine the round trip time.

In [ ]:
# update policy
az sql server conn-policy update --connection-type Proxy
# confirm update
az sql server conn-policy show

If you want to test round trip time, you can connect with SSMS, create a new query (below), and choose to "Include Client Statistics" in your results. Be sure to right-click on the database and select **New Query**. In the results, the "Wait time on server replies" is the best indicator of network latency. You can run this a few times to get a good average.  

```sql
-- Proxy
SELECT * FROM SalesLT.Product
GO 20
```

After 10 trials, I had an average wait time on server replies of `46.6000`

What if we want to make everything `Redirect` so we can attempt to achieve reduced latency?

First, for anything that is **outside Azure**, you need to allow inbound and outbound communication on ports in the range of 11000 - 11999. This is required for the Redirect connection policy. Since you are connecting through an Azure VM, there is no action here.  

Update the connection policy and confirm that update with the following two commands.  

In [ ]:
# update policy
az sql server conn-policy update --connection-type Redirect
# confirm update
az sql server conn-policy show

Now, to test network latency from the `Redirect` policy, connect with SSMS, create a new query (below), and choose to "Include Client Statistics" in your results. Compare the "Wait time on server replies" with your query for `Proxy`.  

> Note: you'll need to create a **new connection to query** to evaluate (i.e. right-click on the Adventure Works database and select New Query, do not use the query window you used to test Proxy).

```sql
-- Redirect
SELECT * FROM SalesLT.Product
GO 20
```

After 10 trials, I have an average wait time on server replies of `25.8000`, which is almost half that of the Proxy connection policy.   

### To review

Redirect is faster because after the intial connection, you can bypass the gateway and go straight to the database. This means less hops, which results in less latency, which ultimately helps in preventing bottlenecks (especially important for chatty applications).



Set it back to default for future activities, by using the same set of commands.

In [ ]:
# update policy
az sql server conn-policy update --connection-type Default
# confirm update
az sql server conn-policy show